In [305]:
import sys
sys.path += ["../src"]
import utils
from glob import glob
import matplotlib.pyplot as plt 
import seaborn as sns
from matplotlib.pyplot import subplots as sbp 
from importlib import reload
import jl_vae
import jl_nflows_geo_coordinates_2 as nfg
from jl_nflows_geo_coordinates import load_nf as load_dict

from _51_abm_functions import cod_prov_abbrv_df

# Global Spatial Autocorrelation
from spatial_autocorrelation import get_moransI, moransI_scatterplot, hypothesis_testing
# Local Spatial Autocorrelation
from spatial_autocorrelation import get_localMoransI, LISA_scatterplot
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.preprocessing import OneHotEncoder
from jl_synthetic_pop_all_provinces import add_cat_features
from ipfn import ipfn
import config


In [298]:
features_synpop = ['flag_garage', 'flag_pertinenza', 'flag_air_conditioning',
            'flag_multi_floor', 'log_mq', 'ANNO_COSTRUZIONE_1500_1965',
            'ANNO_COSTRUZIONE_1965_1985', 'ANNO_COSTRUZIONE_1985_2005',
            'ANNO_COSTRUZIONE_2005_2025', 'ANNO_COSTRUZIONE_Missing',
            'High_energy_class', 'Low_energy_class', 'Medium_energy_class',
            'Missing_energy_class', 'COD_CAT_A02', 'COD_CAT_A03',
            'COD_CAT_A_01_07_08', 'COD_CAT_A_04_05', 'floor_0.0', 'floor_1.0',
            'floor_2.0', 'floor_3.0', 'floor_Missing', 'floor_plus_4',
            'flag_air_conditioning_Missing', 'flag_multi_floor_Missing', 'y', 'x','log_price']

def transform_bins(df, vars):
    df1 = df.copy()
    for var in vars:
        _, bins_values =  pd.qcut(df1[var], q = 5, labels = False, duplicates = "drop", retbins = True)
        bins_values[0] = -np.inf
        bins_values[-1] = np.inf
        df1[var + "_bin"] = pd.cut(df1[var], bins = bins_values, labels = np.arange(len(bins_values) - 1)).astype(int)
        df1.drop(columns = [var], inplace = True)
    return df1

def transform_bins_to_real(sample_df_, df_real, vars):
    sample_df = sample_df_.copy()

    for var in vars:
        _, bins_values = pd.qcut(df_real[var], q = 5, labels = False, duplicates = "drop", retbins = True)
        np.random.seed(2)
        sample_df[var] = [np.random.uniform(low = bins_values[u], high = bins_values[u + 1]) for u in sample_df[var + "_bin"]]
        sample_df.drop(columns = [var + "_bin"], inplace = True)
    return sample_df    


In [299]:
prov = "AL"

In [300]:
shuffle = 0

In [301]:
geo_dict = jl_vae.load_geo_data()
df_real = pd.read_csv(f"/data/housing/data/intermediate/jl_pop_synth/real_populations/df_real_{prov}.csv", index_col = 0)[features_synpop + ["CAP"]]
if True:
            df_real95 = df_real.sample(frac = 0.95, random_state = 1111)

            np.random.seed(2)            
            ran_xy = np.random.uniform(low = df_real["x"].min(), high = df_real["x"].max(), size = 1000000), np.random.uniform(low = df_real["y"].min(), high = df_real["y"].max(), size = 1000000)

            df_locations = utils.spatial_matching_ABM(pd.DataFrame({"x": ran_xy[0], "y": ran_xy[1]}), geo_dict["hydro_risk"], geo_dict["census"], 
                                                         geo_dict["omi_og"], geo_dict["cap"]).query("prov_abbrv == @prov")
            
            

            sample_dfs = []
            for df_ in [df_real.copy(), df_real95.copy()]:
                df_ = df_.dropna(subset = "CAP").assign(CAP = lambda x: [f"{int(u):05d}" for u in x["CAP"]])
                df_.drop(columns = ["x", "y"], inplace = True)
            
                df_sample_list = []
            
                for cap in df_["CAP"].unique():
                    if int(shuffle):
                        locations_cap = df_locations
                    else:
                        locations_cap = df_locations.query("CAP == @cap")
                    df_cap = df_.query("CAP == @cap").dropna()
                    N_cap = len(df_cap)
                    if N_cap > 1:
                        df_cap = transform_bins(df_cap, ["log_mq", "log_price"])            
                        sample_df_cap = df_cap.sample(n = 10 * N_cap,
                                                    random_state = 2, replace = True)

                        sample_df_cap = transform_bins_to_real(sample_df_cap, df_, ["log_mq", "log_price"])
                    else:
                        sample_df_cap = pd.concat([df_cap] * 10)
                    
                    if len(locations_cap) > 0:
                        cap_locs = locations_cap.sample(n = 10 * N_cap, replace = True, random_state = 2)
                        sample_df_cap["x"] = list(cap_locs["GEO_LONGITUDINE_BENE_ROUNDED"])
                        sample_df_cap["y"] = list(cap_locs["GEO_LATITUDINE_BENE_ROUNDED"])
                        sample_df_cap["CAP"] = cap
                    
                    df_sample_list.append(sample_df_cap)
                sample_df = pd.concat(df_sample_list)
                sample_dfs.append(sample_df)

ValueError: Cannot convert float NaN to integer

In [283]:
real_df1 = df_real.dropna().sample(1000, random_state = 12)

In [284]:
real_bins1 = transform_bins(real_df1, ["log_mq", "log_price"])

In [296]:
a = 1
if a:
    print("blue")

blue


In [290]:
real_bins1.columns

Index(['flag_garage', 'flag_pertinenza', 'flag_air_conditioning',
       'flag_multi_floor', 'ANNO_COSTRUZIONE_1500_1965',
       'ANNO_COSTRUZIONE_1965_1985', 'ANNO_COSTRUZIONE_1985_2005',
       'ANNO_COSTRUZIONE_2005_2025', 'ANNO_COSTRUZIONE_Missing',
       'High_energy_class', 'Low_energy_class', 'Medium_energy_class',
       'Missing_energy_class', 'COD_CAT_A02', 'COD_CAT_A03',
       'COD_CAT_A_01_07_08', 'COD_CAT_A_04_05', 'floor_0.0', 'floor_1.0',
       'floor_2.0', 'floor_3.0', 'floor_Missing', 'floor_plus_4',
       'flag_air_conditioning_Missing', 'flag_multi_floor_Missing', 'y', 'x',
       'CAP', 'log_mq_bin', 'log_price_bin'],
      dtype='object')

In [285]:
real_df2 = transform_bins_to_real(real_bins1, real_df1, ["log_mq", "log_price"])

In [286]:
bins_mq = pd.qcut(real_df1["log_mq"], q = 5, labels = False, duplicates = "drop", retbins = True)[1]
bins_price = pd.qcut(real_df1["log_price"], q = 5, labels = False, duplicates = "drop", retbins = True)[1]

bins_mq[0] = -np.inf
bins_mq[-1] = np.inf
bins_price[0] = -np.inf
bins_price[-1] = np.inf
        

In [291]:
real_bins2 = real_df2.copy()
real_bins2["log_mq_bin"] = pd.cut(real_df2["log_mq"], bins = bins_mq, labels = np.arange(5))
real_bins2["log_price_bin"] = pd.cut(real_df2["log_price"], bins = bins_price, labels = np.arange(5))
                      

In [293]:
(real_bins1["log_mq_bin"] == real_bins2["log_mq_bin"] ).sort_values()

2679    True
2486    True
2464    True
868     True
1164    True
        ... 
2649    True
1961    True
789     True
2469    True
408     True
Name: log_mq_bin, Length: 1000, dtype: bool

In [274]:
k,var1, var2 = 1902, "price_bin", "log_price"
real_bins2.loc[k,var1], real_bins1.loc[k,var1]

(3, 4)

In [275]:
real_df1.loc[k,var2], real_df2.loc[k,var2]

(11.982929094215963, 11.80204299795427)

In [276]:
bins_price

array([       -inf, 11.1844214 , 11.46163217, 11.71504165, 11.99431336,
               inf])

In [277]:
geo_dict = jl_vae.load_geo_data()
date = "250718"

prov = "PE"

df_real = pd.read_csv(f"/data/housing/data/intermediate/jl_pop_synth/real_populations/df_real_{prov}.csv", index_col = 0)[features_synpop + ["CAP"]]
df_real95 = df_real.sample(frac = 0.95, random_state = 1111)

sample_locs = True
np.random.seed(2)

ran_xy = np.random.uniform(low = df_real["x"].min(), high = df_real["x"].max(), size = 1000000), np.random.uniform(low = df_real["y"].min(), high = df_real["y"].max(), size = 1000000)

df_locations = utils.spatial_matching_ABM(pd.DataFrame({"x": ran_xy[0], "y": ran_xy[1]}), geo_dict["hydro_risk"], geo_dict["census"], 
                                                         geo_dict["omi_og"], geo_dict["cap"]).query("prov_abbrv == @prov")
            
            

sample_dfs = []
for df_ in [df_real.copy(), df_real95.copy()]:
    df_ = df_.dropna(subset = "CAP").assign(CAP = lambda x: [f"{int(u):05d}" for u in x["CAP"]])
    df_.drop(columns = ["x", "y"], inplace = True)

    df_sample_list = []

    for cap in df_["CAP"].unique():
        locations_cap = df_locations.query("CAP == @cap")
        df_cap = df_.query("CAP == @cap").dropna()
        N_cap = len(df_cap)
        if N_cap > 1:
            df_cap = transform_bins(df_cap, ["log_mq", "log_price"])            
            #df_counts = df_cap.value_counts().reset_index()
            #sample_df_cap = df_counts.sample(n = 10 * N_cap, weights = df_counts["count"],
            #                            random_state = 2, replace = True).drop(columns = ["count"])
            sample_df_cap = df_cap.sample(n = 10 * N_cap,
                                        random_state = 2, replace = True)

            sample_df_cap = transform_bins_to_real(sample_df_cap, df_, "log_mq")
            sample_df_cap = transform_bins_to_real(sample_df_cap, df_, "log_price")
        else:
            sample_df_cap = pd.concat([df_cap] * 10)
        
        if len(locations_cap) > 0:
            cap_locs = locations_cap.sample(n = 10 * N_cap, replace = True, random_state = 2)
            sample_df_cap["x"] = list(cap_locs["GEO_LONGITUDINE_BENE_ROUNDED"])
            sample_df_cap["y"] = list(cap_locs["GEO_LATITUDINE_BENE_ROUNDED"])
            sample_df_cap["CAP"] = cap
                    
        df_sample_list.append(sample_df_cap)
    sample_df = pd.concat(df_sample_list)
    sample_dfs.append(sample_df)
            #sample_dfs[0].to_csv(jl_vae.path_pop_synth + f"ipf_samples/df_sample_{prov}_{date}.csv")
            #sample_dfs[1].to_csv(jl_vae.path_pop_synth + f"ipf_samples/df_sample95_{prov}_{date}.csv")
        #except Exception as e:
        #    print(e)   

KeyError: 'l'

In [53]:
df_real.dropna(subset  ="CAP", inplace = True)

In [37]:
df_ipf = pd.concat(sample_dfs).sample(len(df_real), random_state =1)[df_real.columns]

In [54]:
df_real["CAP"] = [f"{int(u):5d}" for u in df_real["CAP"].dropna()]

In [55]:
var = "log_price"
bins_values_price = pd.qcut(df_real[var], q = 5, labels = False, duplicates = "drop", retbins = True)[1]
var = "log_mq"
bins_values_mq = pd.qcut(df_real[var], q = 5, labels = False, duplicates = "drop", retbins = True)[1]
bins_values_mq[0] = -np.inf
bins_values_mq[-1] = np.inf
bins_values_price[0] = -np.inf
bins_values_price[-1] = np.inf

In [56]:
df_ipf["price_bin"] = pd.cut(df_ipf["log_price"], bins = bins_values_price, labels = np.arange(5))
df_ipf["mq_bin"] = pd.cut(df_ipf["log_mq"], bins = bins_values_mq, labels = np.arange(5))

df_real["price_bin"] = pd.cut(df_real["log_price"], bins = bins_values_price, labels = np.arange(5))
df_real["mq_bin"] = pd.cut(df_real["log_mq"], bins = bins_values_mq, labels = np.arange(5))

In [57]:
obs_ipf = df_ipf.drop(columns = ["x", "y", "log_mq", "log_price"]).drop_duplicates()
obs_real = df_real.drop(columns = ["x", "y", "log_mq", "log_price"]).drop_duplicates()

In [58]:
real_list = [list(u) for _,u in obs_real.iterrows()]
ipf_list = [list(u) for _,u in obs_ipf.iterrows()]

In [59]:
np.mean([u in real_list for u in ipf_list])

0.5615499254843517

In [74]:
var = ["mq_bin", "CAP"]
real_vars = [list(k) for _,k in obs_real[var].iterrows()]
[list(u) for _,u in obs_ipf[var].iterrows() if list(u) not in real_vars]

[[3, '73053'],
 [0, '73027'],
 [0, '73027'],
 [0, '73033'],
 [0, '73021'],
 [0, '73021'],
 [0, '73021'],
 [0, '73033'],
 [0, '73027'],
 [2, '73035'],
 [0, '73021'],
 [0, '73021'],
 [0, '73033'],
 [0, '73027'],
 [0, '73021'],
 [0, '73021'],
 [0, '73021'],
 [0, '73021'],
 [0, '73021'],
 [0, '73021'],
 [1, '73032'],
 [1, '73038'],
 [0, '73034'],
 [0, '73021'],
 [0, '73033'],
 [0, '73034'],
 [0, '73027']]

In [79]:
df_real.query("CAP == '73021'")["mq_bin"].unique()

[3, 4, 2, 1]
Categories (5, int64): [0 < 1 < 2 < 3 < 4]

In [8]:
def transform_bins(df, vars):
    df1 = df.copy()
    for var in vars:
        df1[var + "_bin"] =  pd.qcut(df1[var], q = 5, labels = False, duplicates = "drop").astype(int)
        df1.drop(columns = [var], inplace = True)
    return df1

def transform_bins_to_real(sample_df, df_real, var):
    bins_values = pd.qcut(df_real[var], q = 5, labels = False, duplicates = "drop", retbins = True)[1]
    np.random.seed(2)
    sample_df[var] = [np.random.uniform(low = bins_values[u], high = bins_values[u + 1]) for u in sample_df[var + "_bin"]]
    sample_df.drop(columns = [var + "_bin"], inplace = True)
    return sample_df

,flag_garage,flag_pertinenza,flag_air_conditioning,flag_multi_floor,ANNO_COSTRUZIONE_1500_1965,ANNO_COSTRUZIONE_1965_1985,ANNO_COSTRUZIONE_1985_2005,ANNO_COSTRUZIONE_2005_2025,ANNO_COSTRUZIONE_Missing,High_energy_class,...,floor_3.0,floor_Missing,floor_plus_4,flag_air_conditioning_Missing,flag_multi_floor_Missing,CAP,log_mq,log_price,x,y
2709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,73042,4.429187,11.428967,18.180461,40.005726
988,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,73013,4.973095,11.404882,18.150013,40.228044
1535,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,73043,5.035195,11.481500,18.078784,40.286474
846,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,73014,4.437938,11.430525,18.006776,39.995961
1860,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,73100,6.147716,13.154855,18.196046,40.391583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,73059,4.218692,11.896838,18.171686,39.878434
3733,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,73100,4.035226,11.605566,18.288893,40.386309
6196,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,73021,5.344114,10.433140,18.281930,40.259493
4131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,73020,5.166868,13.077953,18.270547,40.268776
